### 具体的に比較する対象
- click
- fire

### 選定基準
- 学習コスト低い
- サブコマンドなどつくりやすい(例: `trian_predict.py` から train()とpredict()の両方を呼び出せるようにしたい)
- ヘルプがつけられる

### 比較から除外したもの
- argparse : サブコマンドなど使いづらい&clickが上位互換だと感じたため
- docopt : docstring自体のデバッグがしづらい&docstringがhelpになるfireに優位性を感じたため
- Baker : 現在コードが非公開＆開発が止まっていそうなため

# Code

In [1]:
!cat hello_click.py

from typing import Tuple

import click


@click.group()
def cmd():
    pass


@cmd.command()
@click.option("--name", type=str, default="world", help="echo name", show_default=True)
def hello(name: str):
    print(f"hello {name}")


@cmd.command()
@click.argument("values", type=int, required=True, nargs=-1)
def add(values: Tuple[int]):
    print(f"result {sum(values)}")


def main():
    cmd()


if __name__ == "__main__":
    main()


In [2]:
!cat hello_fire.py

from typing import Tuple

import fire


class Test:
    def hello(self, name: str = "world"):
        """
        name: echo name
        """
        print(f"hello {name}")

    def add(self, values: Tuple[int]):
        """
        values: list sum values
        """
        print(f"result {sum(values)}")


if __name__ == "__main__":
    fire.Fire(Test)


# Help

## click
- ヘルプ追加方法：デコレータ内で `help=hoge` で記述
- デフォルト値: `@click.option` 内で設定、 `show_default=True` でヘルプに表示
- 関数アノテーションはヘルプ：表示してくれる
- optionに対してはヘルプ追加可能だが、argumentには追加できない

In [3]:
! poetry run python hello_click.py --help

Usage: hello_click.py [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  add
  hello


In [4]:
! poetry run python hello_click.py hello --help

Usage: hello_click.py hello [OPTIONS]

Options:
  --name TEXT  echo name  [default: world]
  --help       Show this message and exit.


In [5]:
! poetry run python hello_click.py add --help

Usage: hello_click.py add [OPTIONS] VALUES...

Options:
  --help  Show this message and exit.


## fire
- ヘルプはdocstring内に `変数名: ヘルプ内容` で設定可能
- ヘルプの表示コマンドが少し特殊(特にサブコマンドも含める場合) `*.py - -- --help`
- デフォルト値：メソッドの引数で定義、定義されている場合そのコマンドのhelpを呼べば自動で表示
- 関数アノテーション：ヘルプに表示してくれる

In [6]:
! poetry run python hello_fire.py - -- --help

NAME
    hello_fire.py

SYNOPSIS
    hello_fire.py - COMMAND

COMMANDS
    COMMAND is one of the following:

     add
       values: list sum values

     hello
       name: echo name


In [7]:
! poetry run python hello_fire.py add --help

INFO: Showing help with the command 'hello_fire.py add -- --help'.

NAME
    hello_fire.py add - values: list sum values

SYNOPSIS
    hello_fire.py add VALUES

DESCRIPTION
    values: list sum values

POSITIONAL ARGUMENTS
    VALUES
        Type: typing.Tuple[int]

NOTES
    You can also use flags syntax for POSITIONAL ARGUMENTS


In [8]:
! poetry run python hello_fire.py hello --help

INFO: Showing help with the command 'hello_fire.py hello -- --help'.

NAME
    hello_fire.py hello - name: echo name

SYNOPSIS
    hello_fire.py hello <flags>

DESCRIPTION
    name: echo name

FLAGS
    --name=NAME
        Type: str
        Default: 'world'


# 型チェック

## click
- デコレータでtypeを指定している場合、チェックしてくれる

In [9]:
!poetry run python hello_click.py add 1 one 3

Usage: hello_click.py add [OPTIONS] VALUES...
Try 'hello_click.py add --help' for help.

Error: Invalid value for 'VALUES...': one is not a valid integer


## fire
- 型チェック機能なし

In [10]:
!poetry run python hello_fire.py add 1 one 3

Traceback (most recent call last):
  File "hello_fire.py", line 21, in <module>
    fire.Fire(Test)
  File "/Users/yusukehoribe/Library/Caches/pypoetry/virtualenvs/test-command-line-parser-y7NbsANJ-py3.8/lib/python3.8/site-packages/fire/core.py", line 141, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/Users/yusukehoribe/Library/Caches/pypoetry/virtualenvs/test-command-line-parser-y7NbsANJ-py3.8/lib/python3.8/site-packages/fire/core.py", line 466, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
  File "/Users/yusukehoribe/Library/Caches/pypoetry/virtualenvs/test-command-line-parser-y7NbsANJ-py3.8/lib/python3.8/site-packages/fire/core.py", line 681, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
  File "hello_fire.py", line 17, in add
    print(f"result {sum(values)}")
TypeError: 'int' object is not iterable


# テストしやすさ？

## click
- scriptからは呼び出せなくなる
- テストには専用のCliRunnerを使う
　- `asert result.exit_code == 0` などで正常実行されたかチェックできる
　- 正常実行されたが、outputが正しくない場合のテストでの検出がちょっと難しそう？ 

In [11]:
from hello_click import hello as hello_cl
from hello_click import add as add_cl

In [12]:
hello_cl(name="capella")

TypeError: __init__() got an unexpected keyword argument 'name'

In [13]:
from click.testing import CliRunner

runner = CliRunner()
result = runner.invoke(add_cl, ["1","one","2"])

In [14]:
# 0: 正常終了
print(f"exit_code: {result.exit_code}")
print(f"exc_info: {result.exc_info}")
print(f"exception: {result.exception}")
print(f"output: {result.output}")

exit_code: 2
exc_info: (<class 'SystemExit'>, SystemExit(2), <traceback object at 0x10f7ad180>)
exception: 2
output: Usage: add [OPTIONS] VALUES...
Try 'add --help' for help.

Error: Invalid value for 'VALUES...': one is not a valid integer



## fire
- scriptとして普通に呼び出せる

In [15]:
from hello_fire import Test

In [16]:
Test().add(values=(1,'one',3))

TypeError: unsupported operand type(s) for +: 'int' and 'str'

# まとめ
- 運用時の引数に対する型チェック必要、cli以外から呼び出さないなら：click
- 運用時の引数に対する型チェック不要、関数アノテーション/docstring/assertionなどを実装するルールにするなら：fire